# 1.1.1 文書分類

In [3]:
from transformers import pipeline

/workspaces/transformers-tutorials/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text_classification_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-marc_ja")
positive_text = "世界には言葉がわからなくても感動する音楽がある。"

# positive_textの極性を予測
print(text_classification_pipeline(positive_text)[0])

{'label': 'positive', 'score': 0.9993619322776794}


In [4]:
negative_text = "世界には言葉に出ないほどひどい音楽がある。"
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9855380058288574}


# 1.1.2 自然言語推論

In [5]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text = "二人の男性がジェット機を見ています。"
entailment_text = "ジェット機を見ている二人の男性がいます。"

# 理論関係を予測
print(nli_pipeline({"text": text, "text_pair": entailment_text}))

{'label': 'entailment', 'score': 0.9977477192878723}


In [12]:
contradiction_text = "二人の男性がが飛んでいます。"

# 矛盾関係を予測
print(nli_pipeline({"text": text, "text_pair": contradiction_text}))

{'label': 'neutral', 'score': 0.9984857439994812}


In [10]:
neutral_text = "二人の男性が、白い飛行機を眺めています。"

# 中立関係を予測
print(nli_pipeline({"text": text, "text_pair": neutral_text}))

{'label': 'neutral', 'score': 0.9976142644882202}


# 1.1.3 意味的類似度計算

In [5]:
text_slim_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jsts", function_to_apply="none")

text = "川縁でサーフを持った人たちがいます。"
sim_text = "サーファーたちが川縁に立っています。"

# 類似度を予測
result = text_slim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

3.8350539207458496


In [6]:
dissim_text = "トイレの壁に黒いタオルがかけられています。"

# 類似度を予測
result = text_slim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

0.04958426207304001


In [8]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki", task="feature-extraction")

# ベクトルの取得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]

# コサイン類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.8406088352203369


In [11]:
# ベクトルの取得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]

# コサイン類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.3928905725479126


# 1.1.4 固有表現認識

In [16]:
from pprint import pprint

ner_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset", aggregation_strategy="simple")

text = "大谷翔平は岩手県水沢市出身のプロ野球選手"

# 固有表現抽出
pprint(ner_pipeline(text))

[{'end': None,
  'entity_group': '人名',
  'score': np.float32(0.99823624),
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': np.float32(0.9986874),
  'start': None,
  'word': '岩手 県 水沢 市'}]


# 1.1.5 要約生成

In [21]:
text2text_pipeline = pipeline(model="llm-book/t5-base-long-livedoor-news-corpus")

article = "ついに始まった３連休。テレビを見ながら過ごしている人も多いのではないだろうか？　今夜オススメなのは何と言っても、NHKスペシャル「世界を変えた男 スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。 今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界でのジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。 世界を変えた男　スティーブ・ジョブズ（NHKスペシャル）"

# 要約を生成
pprint(text2text_pipeline(article)[0]["generated_text"])

'今夜はNHKスペシャル「世界を変えた男 スティーブ・ジョブズ」をチェック!'


# 1.2 transformersの基本的な使い方

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")

tokenizer.tokenize("今日は天気がいいので")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


['▁', '今日', 'は', '天気', 'が', 'いい', 'の', 'で']

In [23]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("abeja/gpt2-large-japanese")

inputs = tokenizer("今日は天気がいいので", return_tensors="pt")

outputs = model.generate(**inputs, max_length=15, pad_token_id=tokenizer.pad_token_id)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

今日は天気がいいので外でお弁当を食べました
